In [ ]:
import pandas as pd 
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split




In [35]:
bbb = pd.read_csv("BBB-1.csv")

In [36]:
bbb

,acctnum,gender,state,zip,zip3,first,last,book$,nonbook$,total$,purch,child,youth,cook,do_it,refernce,art,geog,buyer
0,10001,M,NY,10605,106,49,29,109,248,357,10,3,2,2,0,1,0,2,0
1,10002,M,NY,10960,109,39,27,35,103,138,3,0,1,0,1,0,0,1,0
2,10003,F,PA,19146,191,19,15,25,147,172,2,0,0,2,0,0,0,0,0
3,10004,F,NJ,7016,70,7,7,15,257,272,1,0,0,0,0,1,0,0,0
4,10005,F,NY,10804,108,15,15,15,134,149,1,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,59996,F,NY,11967,119,9,9,15,12,27,1,1,0,0,0,0,0,0,0
49996,59997,F,NJ,8882,88,25,5,79,294,373,7,3,0,1,1,0,1,1,0
49997,59998,M,NJ,7410,74,3,3,15,178,193,1,0,0,0,0,1,0,0,0
49998,59999,M,NJ,7090,70,49,29,98,246,344,8,2,0,1,0,2,1,2,0


Step 1 — Explore the BBB dataset
Before we run any regressions, start by exploring the data. Look at the different variables and think about what kind of information they contain. Which variables are numeric (like last, purch, total$)? Which are categorical (like gender, product categories)? Our outcome of interest is buyer, coded 1 if the customer bought the Florence book and 0 otherwise. This matters because the type of regression we choose depends on whether the outcome is continuous or binary. Compute simple descriptive statistics: what is the mean and median of total$? What percentage of customers purchased (the overall buyer rate)? How does this break down by gender, or by product category? These quick checks give us a baseline intuition before we model.

In [37]:
# Mean of total spending across all customers
total_mean = bbb['total$'].mean()

# Median of total spending (helps detect skewness)
total_median = bbb['total$'].median()

print(total_mean, total_median)


208.31832 209.0


In [38]:
# Since buyer is coded as 0/1, the mean equals the percentage who purchased
bbb['buyer'].mean()

# Average buyer rate for males vs females
avg_buyer = bbb.groupby('gender')['buyer'].mean()

# Include counts to see sample sizes by gender
avg_gender = bbb.groupby('gender')['buyer'].agg(['mean', 'count'])

print(avg_buyer, avg_gender)

gender
F    0.071737
M    0.127740
Name: buyer, dtype: float64             mean  count
gender                 
F       0.071737  33302
M       0.127740  16698


In [39]:
# Compare average total spending for buyers vs non-buyers
spending_by_buyer_mean =bbb.groupby('buyer')['total$'].mean()

# Compare median spending (more robust to outliers)
spending_by_buyer_median = bbb.groupby('buyer')['total$'].median()

As a first experiment, fit a linear regression using buyer (0/1) as the dependent variable and one predictor, such as purch. This is technically the wrong model for binary outcomes, but it is useful to see why. You will notice that predictions are not confined between 0 and 1, and coefficients are not easy to interpret in terms of probabilities. This illustrates why linear regression is not suited for yes/no outcomes.

In [40]:
bbb.columns

Index(['acctnum', 'gender', 'state', 'zip', 'zip3', 'first', 'last', 'book$',
       'nonbook$', 'total$', 'purch', 'child', 'youth', 'cook', 'do_it',
       'refernce', 'art', 'geog', 'buyer'],
      dtype='object')

In [41]:
# makes into numric
bbb['buyer'] = pd.to_numeric(bbb['buyer'])
bbb['purch'] = pd.to_numeric(bbb['purch'])


y = bbb['buyer']
# needs 2 brackoets cuz its 2d matrix 
X = bbb[['purch']]

In [42]:
bbb.dtypes


acctnum      int64
gender      object
state       object
zip          int64
zip3         int64
first        int64
last         int64
book$        int64
nonbook$     int64
total$       int64
purch        int64
child        int64
youth        int64
cook         int64
do_it        int64
refernce     int64
art          int64
geog         int64
buyer        int64
dtype: object

In [43]:
# Initialize the model
model = LinearRegression()

# Fitting model
model.fit(X,y)

,"fit_intercept fit_intercept: bool, default=TrueWhether to calculate the intercept for this model. If setto False, no intercept will be used in calculations(i.e. data is expected to be centered).",True
,"copy_X copy_X: bool, default=TrueIf True, X will be copied; else, it may be overwritten.",True
,"tol tol: float, default=1e-6The precision of the solution (`coef_`) is determined by `tol` whichspecifies a different convergence criterion for the `lsqr` solver.`tol` is set as `atol` and `btol` of :func:`scipy.sparse.linalg.lsqr` whenfitting on sparse training data. This parameter has no effect when fittingon dense data... versionadded:: 1.7",1e-06
,"n_jobs n_jobs: int, default=NoneThe number of jobs to use for the computation. This will only providespeedup in case of sufficiently large problems, that is if firstly`n_targets > 1` and secondly `X` is sparse or if `positive` is setto `True`. ``None`` means 1 unless in a:obj:`joblib.parallel_backend` context. ``-1`` means using allprocessors. See :term:`Glossary ` for more details.",None
,"positive positive: bool, default=FalseWhen set to ``True``, forces the coefficients to be positive. Thisoption is only supported for dense arrays.For a comparison between a linear regression model with positive constraintson the regression coefficients and a linear regression without such constraints,see :ref:`sphx_glr_auto_examples_linear_model_plot_nnls.py`... versionadded:: 0.24",False


In [44]:
# Predicted values (some may be <0 or >1)
y_hat = model.predict(X)

y_hat.min(), y_hat.max()


(np.float64(0.06169411896157094), np.float64(0.17109917858427087))

In [49]:
# Starter code: Linear regression with a binary outcome
import statsmodels.api as sm

X = bbb[['purch']]
y = bbb['buyer']

X = sm.add_constant(X)  # add intercept
model_lin = sm.OLS(y, X).fit()
print(model_lin.summary())

# Predicted values (note some may be <0 or >1)
bbb['lin_pred'] = model_lin.predict(X)
print(bbb[['purch', 'buyer', 'lin_pred']].head(10))


                            OLS Regression Results                            
Dep. Variable:                  buyer   R-squared:                       0.015
Model:                            OLS   Adj. R-squared:                  0.015
Method:                 Least Squares   F-statistic:                     737.3
Date:                Thu, 05 Feb 2026   Prob (F-statistic):          3.50e-161
Time:                        17:05:57   Log-Likelihood:                -8134.4
No. Observations:               50000   AIC:                         1.627e+04
Df Residuals:                   49998   BIC:                         1.629e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0517      0.002     27.079      0.0

In [45]:
model.coef_

array([0.00994591])

In [46]:
model.intercept_

np.float64(0.051748204450416405)

I fit a linear regression model with buyer as the dependent variable and purch as the predictor. Although the model runs successfully, the linear framework is not appropriate for a binary outcome because predicted values are not constrained to lie between 0 and 1 and coefficients are difficult to interpret as probabilities.

Step 3 — Logistic regression with one variable
Now fit a logistic regression with buyer as the dependent variable and purch as the predictor. Logistic regression models the log-odds of purchase, but you can interpret results in terms of predicted probabilities. These predicted probabilities always lie between 0 and 1, making them meaningful for targeting. Choose a cutoff based on the break-even rate: recall from RFM that the mailing cost is $0.50 per customer and the profit per sale is $6, so the break-even response rate is 0.083 (8.3%). Target customers with predicted purchase probability greater than or equal to this threshold. For those customers, calculate the campaign economics: cost = 0.50 times the number mailed, sales = 6 times the number of actual buyers in that group, profit = sales minus cost, and ROME = profit divided by cost.

In [53]:
from sklearn.linear_model import LogisticRegression

# Define outcome and predictor
y = bbb['buyer']
X = bbb[['purch']]

# Fit logistic regression
logit = LogisticRegression(max_iter=1000)
logit.fit(X, y)


,"penalty penalty: {'l1', 'l2', 'elasticnet', None}, default='l2'Specify the norm of the penalty:- `None`: no penalty is added;- `'l2'`: add a L2 penalty term and it is the default choice;- `'l1'`: add a L1 penalty term;- `'elasticnet'`: both L1 and L2 penalty terms are added... warning:: Some penalties may not work with some solvers. See the parameter `solver` below, to know the compatibility between the penalty and solver... versionadded:: 0.19 l1 penalty with SAGA solver (allowing 'multinomial' + L1).. deprecated:: 1.8 `penalty` was deprecated in version 1.8 and will be removed in 1.10. Use `l1_ratio` instead. `l1_ratio=0` for `penalty='l2'`, `l1_ratio=1` for `penalty='l1'` and `l1_ratio` set to any float between 0 and 1 for `'penalty='elasticnet'`.",'deprecated'
,"C C: float, default=1.0Inverse of regularization strength; must be a positive float.Like in support vector machines, smaller values specify strongerregularization. `C=np.inf` results in unpenalized logistic regression.For a visual example on the effect of tuning the `C` parameterwith an L1 penalty, see::ref:`sphx_glr_auto_examples_linear_model_plot_logistic_path.py`.",1.0
,"l1_ratio l1_ratio: float, default=0.0The Elastic-Net mixing parameter, with `0 <= l1_ratio <= 1`. Setting`l1_ratio=1` gives a pure L1-penalty, setting `l1_ratio=0` a pure L2-penalty.Any value between 0 and 1 gives an Elastic-Net penalty of the form`l1_ratio * L1 + (1 - l1_ratio) * L2`... warning:: Certain values of `l1_ratio`, i.e. some penalties, may not work with some solvers. See the parameter `solver` below, to know the compatibility between the penalty and solver... versionchanged:: 1.8 Default value changed from None to 0.0... deprecated:: 1.8 `None` is deprecated and will be removed in version 1.10. Always use `l1_ratio` to specify the penalty type.",0.0
,"dual dual: bool, default=FalseDual (constrained) or primal (regularized, see also:ref:`this equation `) formulation. Dual formulationis only implemented for l2 penalty with liblinear solver. Prefer `dual=False`when n_samples > n_features.",False
,"tol tol: float, default=1e-4Tolerance for stopping criteria.",0.0001
,"fit_intercept fit_intercept: bool, default=TrueSpecifies if a constant (a.k.a. bias or intercept) should beadded to the decision function.",True
,"intercept_scaling intercept_scaling: float, default=1Useful only when the solver `liblinear` is usedand `self.fit_intercept` is set to `True`. In this case, `x` becomes`[x, self.intercept_scaling]`,i.e. a ""synthetic"" feature with constant value equal to`intercept_scaling` is appended to the instance vector.The intercept becomes``intercept_scaling * synthetic_feature_weight``... note:: The synthetic feature weight is subject to L1 or L2 regularization as all other features. To lessen the effect of regularization on synthetic feature weight (and therefore on the intercept) `intercept_scaling` has to be increased.",1
,"class_weight class_weight: dict or 'balanced', default=NoneWeights associated with classes in the form ``{class_label: weight}``.If not given, all classes are supposed to have weight one.The ""balanced"" mode uses the values of y to automatically adjustweights inversely proportional to class frequencies in the input dataas ``n_samples / (n_classes * np.bincount(y))``.Note that these weights will be multiplied with sample_weight (passedthrough the fit method) if sample_weight is specified... versionadded:: 0.17 *class_weight='balanced'*",None
,"random_state random_state: int, RandomState instance, default=NoneUsed when ``solver`` == 'sag', 'saga' or 'liblinear' to shuffle thedata. See :term:`Glossary ` for details.",None
,"solver solver: {'lbfgs', 'liblinear', 'newton-cg', 'newton-cholesky', 'sag', 'saga'}, default='lbfgs'Algorithm to use in the optimization problem. Default is 'lbfgs'.To choose a solver, you might want to consider the following aspects:- 'lbfgs' is a good default solver because it works reasonably well for a wide class of problems.- For :term:`mul

In [57]:
# Predicted probability of purchase (P(buyer = 1))
bbb['pred_prob'] = logit.predict_proba(X)[:, 1]

cutoff = 0.083

# Customers to target
bbb['target'] = (bbb['pred_prob'] >= cutoff).astype(int)



In [58]:
# Number of customers mailed
num_mailed = bbb['target'].sum()

# Mailing cost
cost = 0.50 * num_mailed

# Number of actual buyers among targeted customers
num_buyers = bbb.loc[bbb['target'] == 1, 'buyer'].sum()

# Revenue from sales
sales = 6 * num_buyers

# Profit
profit = sales - cost

# Return on Marketing Expenditure (ROME)
rome = profit / cost


In [60]:
print("Number mailed:", num_mailed)
print("Number of buyers:", num_buyers)
print("Campaign cost ($):", cost)
print("Sales revenue ($):", sales)
print("Profit ($):", profit)
print("ROME:", rome)


Number mailed: 17926
Number of buyers: 2346
Campaign cost ($): 8963.0
Sales revenue ($): 14076
Profit ($): 5113.0
ROME: 0.570456320428428


## Step 4 — Logistic regression with multiple variables

Next, expand the model by adding more predictors, such as `last`, `total$`, and `gender`. Each predictor adds more information about purchase behavior. Check the significance of the coefficients and how they affect predicted probabilities.

Recalculate profits and ROME using the same procedure. Compare: does this model outperform the simple one-variable logistic regression?

In [104]:
# Create X with multiple predictors
X_multi = bbb[['purch', 'last', 'total$', 'gender']]

# Convert categorical variable to numeric (dummy variable)
X_multi = pd.get_dummies(X_multi, drop_first=True)

# Outcome
y = bbb['buyer']


KeyError: "['total$'] not in index"

In [ ]:
from sklearn.linear_model import LogisticRegression

logit_multi = LogisticRegression(max_iter=1000)
logit_multi.fit(X_multi, y)


,"penalty penalty: {'l1', 'l2', 'elasticnet', None}, default='l2'Specify the norm of the penalty:- `None`: no penalty is added;- `'l2'`: add a L2 penalty term and it is the default choice;- `'l1'`: add a L1 penalty term;- `'elasticnet'`: both L1 and L2 penalty terms are added... warning:: Some penalties may not work with some solvers. See the parameter `solver` below, to know the compatibility between the penalty and solver... versionadded:: 0.19 l1 penalty with SAGA solver (allowing 'multinomial' + L1).. deprecated:: 1.8 `penalty` was deprecated in version 1.8 and will be removed in 1.10. Use `l1_ratio` instead. `l1_ratio=0` for `penalty='l2'`, `l1_ratio=1` for `penalty='l1'` and `l1_ratio` set to any float between 0 and 1 for `'penalty='elasticnet'`.",'deprecated'
,"C C: float, default=1.0Inverse of regularization strength; must be a positive float.Like in support vector machines, smaller values specify strongerregularization. `C=np.inf` results in unpenalized logistic regression.For a visual example on the effect of tuning the `C` parameterwith an L1 penalty, see::ref:`sphx_glr_auto_examples_linear_model_plot_logistic_path.py`.",1.0
,"l1_ratio l1_ratio: float, default=0.0The Elastic-Net mixing parameter, with `0 <= l1_ratio <= 1`. Setting`l1_ratio=1` gives a pure L1-penalty, setting `l1_ratio=0` a pure L2-penalty.Any value between 0 and 1 gives an Elastic-Net penalty of the form`l1_ratio * L1 + (1 - l1_ratio) * L2`... warning:: Certain values of `l1_ratio`, i.e. some penalties, may not work with some solvers. See the parameter `solver` below, to know the compatibility between the penalty and solver... versionchanged:: 1.8 Default value changed from None to 0.0... deprecated:: 1.8 `None` is deprecated and will be removed in version 1.10. Always use `l1_ratio` to specify the penalty type.",0.0
,"dual dual: bool, default=FalseDual (constrained) or primal (regularized, see also:ref:`this equation `) formulation. Dual formulationis only implemented for l2 penalty with liblinear solver. Prefer `dual=False`when n_samples > n_features.",False
,"tol tol: float, default=1e-4Tolerance for stopping criteria.",0.0001
,"fit_intercept fit_intercept: bool, default=TrueSpecifies if a constant (a.k.a. bias or intercept) should beadded to the decision function.",True
,"intercept_scaling intercept_scaling: float, default=1Useful only when the solver `liblinear` is usedand `self.fit_intercept` is set to `True`. In this case, `x` becomes`[x, self.intercept_scaling]`,i.e. a ""synthetic"" feature with constant value equal to`intercept_scaling` is appended to the instance vector.The intercept becomes``intercept_scaling * synthetic_feature_weight``... note:: The synthetic feature weight is subject to L1 or L2 regularization as all other features. To lessen the effect of regularization on synthetic feature weight (and therefore on the intercept) `intercept_scaling` has to be increased.",1
,"class_weight class_weight: dict or 'balanced', default=NoneWeights associated with classes in the form ``{class_label: weight}``.If not given, all classes are supposed to have weight one.The ""balanced"" mode uses the values of y to automatically adjustweights inversely proportional to class frequencies in the input dataas ``n_samples / (n_classes * np.bincount(y))``.Note that these weights will be multiplied with sample_weight (passedthrough the fit method) if sample_weight is specified... versionadded:: 0.17 *class_weight='balanced'*",None
,"random_state random_state: int, RandomState instance, default=NoneUsed when ``solver`` == 'sag', 'saga' or 'liblinear' to shuffle thedata. See :term:`Glossary ` for details.",None
,"solver solver: {'lbfgs', 'liblinear', 'newton-cg', 'newton-cholesky', 'sag', 'saga'}, default='lbfgs'Algorithm to use in the optimization problem. Default is 'lbfgs'.To choose a solver, you might want to consider the following aspects:- 'lbfgs' is a good default solver because it works reasonably well for a wide class of problems.- For :term:`mul

In [ ]:
coef_table = pd.DataFrame({
    'Variable': X_multi.columns,
    'Coefficient': logit_multi.coef_[0]
})

coef_table


,Variable,Coefficient
0,purch,0.082773
1,last,-0.082697
2,total$,0.000904
3,gender_M,0.509102


In [ ]:
bbb['pred_prob_multi'] = logit_multi.predict_proba(X_multi)[:, 1]

cutoff = 0.083
bbb['target_multi'] = (bbb['pred_prob_multi'] >= cutoff).astype(int)


In [ ]:
num_mailed_multi = bbb['target_multi'].sum()
cost_multi = 0.50 * num_mailed_multi

num_buyers_multi = bbb.loc[bbb['target_multi'] == 1, 'buyer'].sum()
sales_multi = 6 * num_buyers_multi

profit_multi = sales_multi - cost_multi
rome_multi = profit_multi / cost_multi


In [ ]:
print("MULTI-VARIATE LOGISTIC MODEL RESULTS")
print("Number mailed:", num_mailed_multi)
print("Number of buyers:", num_buyers_multi)
print("Campaign cost ($):", round(cost_multi, 2))
print("Sales revenue ($):", round(sales_multi, 2))
print("Profit ($):", round(profit_multi, 2))
print("ROME:", round(rome_multi, 3))


MULTI-VARIATE LOGISTIC MODEL RESULTS
Number mailed: 22473
Number of buyers: 3321
Campaign cost ($): 11236.5
Sales revenue ($): 19926
Profit ($): 8689.5
ROME: 0.773


Yes it outperforms by 0.203

Step 5 — Logistic regression with interaction terms
Introduce the idea of interactions. An interaction allows the effect of one variable to depend on another. For example, you might test gender × art_books. This term captures whether women who buy art books are particularly likely to purchase the Florence book. Fit a model including both main effects and the interaction term. Interpret the coefficients carefully and calculate profits and ROME again. Your task is to identify other categorical variables that can be interacted with main predictors. This shows how personalized targeting (beyond just averages) can lead to higher profitability.

In [ ]:
# Outcome
y = bbb['buyer']

# Ensure key columns are numeric where needed
bbb['purch'] = pd.to_numeric(bbb['purch'])
bbb['total$'] = pd.to_numeric(bbb['total$'])

# Create a numeric gender indicator (Male=1, Female=0)
# If your dataset uses different labels, adjust the mapping accordingly.
bbb['gender_M'] = bbb['gender'].map({'M': 1, 'F': 0})

# Make sure art is numeric (often it's already 0/1)
bbb['art'] = pd.to_numeric(bbb['art'], errors='coerce')

# Interaction term: gender x art
bbb['genderM_x_art'] = bbb['gender_M'] * bbb['art']

# Predictors: include BOTH main effects + interaction + other controls
X_int = bbb[['purch', 'last', 'total$', 'gender_M', 'art', 'genderM_x_art']].copy()

# Drop rows with missing values in predictors/outcome (simple approach)
data = pd.concat([X_int, y], axis=1).dropna()
X_int = data[X_int.columns]
y_clean = data['buyer']


In [ ]:
logit_int = LogisticRegression(max_iter=1000)
logit_int.fit(X_int, y_clean)

coef_table = pd.DataFrame({
    'Variable': X_int.columns,
    'Coefficient': logit_int.coef_[0]
}).sort_values('Coefficient', ascending=False)

coef_table


,Variable,Coefficient
4,art,1.271904
3,gender_M,0.857713
2,total$,0.001048
0,purch,-0.056764
1,last,-0.089476
5,genderM_x_art,-0.345218


In [ ]:
cutoff = 0.083  # 0.50 / 6

data['pred_prob_int'] = logit_int.predict_proba(X_int)[:, 1]
data['target_int'] = (data['pred_prob_int'] >= cutoff).astype(int)


In [ ]:
num_mailed_int = data['target_int'].sum()
cost_int = 0.50 * num_mailed_int

num_buyers_int = data.loc[data['target_int'] == 1, 'buyer'].sum()
sales_int = 6 * num_buyers_int

profit_int = sales_int - cost_int
rome_int = profit_int / cost_int if cost_int > 0 else float('nan')

print("INTERACTION MODEL RESULTS")
print(f"Number mailed: {num_mailed_int}")
print(f"Number of buyers: {int(num_buyers_int)}")
print(f"Campaign cost ($): {cost_int:.2f}")
print(f"Sales revenue ($): {sales_int:.2f}")
print(f"Profit ($): {profit_int:.2f}")
print(f"ROME: {rome_int:.3f}")


INTERACTION MODEL RESULTS
Number mailed: 17016
Number of buyers: 3246
Campaign cost ($): 8508.00
Sales revenue ($): 19476.00
Profit ($): 10968.00
ROME: 1.289


Step 6 — Decision tree for feature importance
Run a decision tree to predict buyer. The tree algorithm will rank variables by their importance in predicting purchase. Look at the feature importance list. Which variables come out on top? Use this ranking to guide which variables (and possible interactions) to include in a refined logistic regression model. This way, the decision tree helps you discover useful predictors, while logistic regression lets you interpret and calculate profitability more directly.

In [ ]:
# Outcome
y = bbb['buyer']

# Predictor set (include rich set)
X_tree = bbb[['purch', 'last', 'total$', 'gender', 'art', 'cook', 'child', 'youth']]

# Convert gender to numeric
X_tree = pd.get_dummies(X_tree, drop_first=True)

# Drop missing values
data_tree = pd.concat([X_tree, y], axis=1).dropna()
X_tree = data_tree[X_tree.columns]
y_tree = data_tree['buyer']


In [ ]:
tree = DecisionTreeClassifier(
    max_depth=4,
    min_samples_leaf=50,
    random_state=42
)

tree.fit(X_tree, y_tree)


,"criterion criterion: {""gini"", ""entropy"", ""log_loss""}, default=""gini""The function to measure the quality of a split. Supported criteria are""gini"" for the Gini impurity and ""log_loss"" and ""entropy"" both for theShannon information gain, see :ref:`tree_mathematical_formulation`.",'gini'
,"splitter splitter: {""best"", ""random""}, default=""best""The strategy used to choose the split at each node. Supportedstrategies are ""best"" to choose the best split and ""random"" to choosethe best random split.",'best'
,"max_depth max_depth: int, default=NoneThe maximum depth of the tree. If None, then nodes are expanded untilall leaves are pure or until all leaves contain less thanmin_samples_split samples.",4
,"min_samples_split min_samples_split: int or float, default=2The minimum number of samples required to split an internal node:- If int, then consider `min_samples_split` as the minimum number.- If float, then `min_samples_split` is a fraction and `ceil(min_samples_split * n_samples)` are the minimum number of samples for each split... versionchanged:: 0.18 Added float values for fractions.",2
,"min_samples_leaf min_samples_leaf: int or float, default=1The minimum number of samples required to be at a leaf node.A split point at any depth will only be considered if it leaves atleast ``min_samples_leaf`` training samples in each of the left andright branches. This may have the effect of smoothing the model,especially in regression.- If int, then consider `min_samples_leaf` as the minimum number.- If float, then `min_samples_leaf` is a fraction and `ceil(min_samples_leaf * n_samples)` are the minimum number of samples for each node... versionchanged:: 0.18 Added float values for fractions.",50
,"min_weight_fraction_leaf min_weight_fraction_leaf: float, default=0.0The minimum weighted fraction of the sum total of weights (of allthe input samples) required to be at a leaf node. Samples haveequal weight when sample_weight is not provided.",0.0
,"max_features max_features: int, float or {""sqrt"", ""log2""}, default=NoneThe number of features to consider when looking for the best split:- If int, then consider `max_features` features at each split.- If float, then `max_features` is a fraction and `max(1, int(max_features * n_features_in_))` features are considered at each split.- If ""sqrt"", then `max_features=sqrt(n_features)`.- If ""log2"", then `max_features=log2(n_features)`.- If None, then `max_features=n_features`... note:: The search for a split does not stop until at least one valid partition of the node samples is found, even if it requires to effectively inspect more than ``max_features`` features.",None
,"random_state random_state: int, RandomState instance or None, default=NoneControls the randomness of the estimator. The features are alwaysrandomly permuted at each split, even if ``splitter`` is set to``""best""``. When ``max_features < n_features``, the algorithm willselect ``max_features`` at random at each split before finding the bestsplit among them. But the best found split may vary across differentruns, even if ``max_features=n_features``. That is the case, if theimprovement of the criterion is identical for several splits and onesplit has to be selected at random. To obtain a deterministic behaviourduring fitting, ``random_state`` has to be fixed to an integer.See :term:`Glossary ` for details.",42
,"max_leaf_nodes max_leaf_nodes: int, default=NoneGrow a tree with ``max_leaf_nodes`` in best-first fashion.Best nodes are defined as relative reduction in impurity.If None then unlimited number of leaf nodes.",None
,"min_impurity_decrease min_impurity_decrease: float, default=0.0A node will be split if this split induces a decrease of the impuritygreater than or equal to this value.The weighted impurity decrease equation is the following:: N_t / N * (impurity - N_t_R / N_t * right_impurity - N_t_L / N_t * left_impurity)where ``N`` is the total number of samples, ``N_t`` is the number ofsamples at the current n

In [ ]:
importance = pd.DataFrame({
    'Variable': X_tree.columns,
    'Importance': tree.feature_importances_
}).sort_values('Importance', ascending=False)

importance


,Variable,Importance
3,art,0.701327
1,last,0.255239
7,gender_M,0.035881
4,cook,0.005803
2,total$,0.001750
0,purch,0.000000
5,child,0.000000
6,youth,0.000000


In [ ]:
X_refined = bbb[['purch', 'last', 'total$', 'art', 'gender']]
X_refined = pd.get_dummies(X_refined, drop_first=True)

# Optional interaction suggested by tree
bbb['genderM_x_art'] = bbb['gender_M'] * bbb['art']
X_refined['genderM_x_art'] = bbb['genderM_x_art']


Part 2 — Enhancing the logistic regression model (graded)

In [ ]:
# Clean and standardize variables used across all models by ensuring numeric types,
# renaming columns for consistency, and removing observations with missing values.
# This creates a common, comparable dataset for RFM, logistic regression, and tree models.


def campaign_metrics(df, target_mask, cost_per_mail=0.50, profit_per_sale=6.00):
    """
    df must contain column 'buyer' (0/1).
    target_mask is a boolean Series indicating who you mail.
    """
    mailed = int(target_mask.sum())
    buyers_mailed = int(df.loc[target_mask, 'buyer'].sum())
    cost = cost_per_mail * mailed
    sales = profit_per_sale * buyers_mailed
    profit = sales - cost
    rome = (profit / cost) if cost > 0 else float('nan')
    return {
        "mailed": mailed,
        "buyers_mailed": buyers_mailed,
        "cost": cost,
        "sales": sales,
        "profit": profit,
        "rome": rome
    }


In [ ]:
# Rename spending column 
if 'total$' in bbb.columns:
    bbb = bbb.rename(columns={'total$': 'total_spent'})

# Ensure numeric values
for col in ['buyer', 'purch', 'last', 'total_spent']:
    if col in bbb.columns:
        bbb[col] = pd.to_numeric(bbb[col], errors='coerce')

# Drop rows missing essentials
df = bbb.dropna(subset=['buyer', 'purch', 'last', 'total_spent']).copy()
df['buyer'] = df['buyer'].astype(int)


Strategy A: RFM targeting (mail only cells with profit > 0)

In [ ]:
cost_per_mail = 0.50
profit_per_sale = 6.00
break_even = cost_per_mail / profit_per_sale  # 0.08333

# Recency: smaller 'last' = more recent. We'll rank so higher = more recent.
df['recency_rank'] = (-df['last']).rank(method='first')
df['R'] = pd.qcut(df['recency_rank'], 5, labels=[1,2,3,4,5]).astype(int)

# Frequency (purch): higher is better
df['F'] = pd.qcut(df['purch'].rank(method='first'), 5, labels=[1,2,3,4,5]).astype(int)

# Monetary (total_spent): higher is better
df['M'] = pd.qcut(df['total_spent'].rank(method='first'), 5, labels=[1,2,3,4,5]).astype(int)

# RFM cell
df['rfm_cell'] = df['R'].astype(str) + df['F'].astype(str) + df['M'].astype(str)

# Compute cell response rate from your actual buyer outcomes
cell_stats = df.groupby('rfm_cell')['buyer'].agg(['mean','count']).rename(columns={'mean':'resp_rate','count':'n'})

# Profit per mailed customer in a cell: 6*resp_rate - 0.50
cell_stats['profit_per_mail'] = profit_per_sale * cell_stats['resp_rate'] - cost_per_mail

# Mail only profitable cells
profitable_cells = cell_stats.index[cell_stats['profit_per_mail'] > 0]
target_rfm = df['rfm_cell'].isin(profitable_cells)

rfm_results = campaign_metrics(df, target_rfm, cost_per_mail, profit_per_sale)
rfm_results


{'mailed': 23471,
 'buyers_mailed': 3276,
 'cost': 11735.5,
 'sales': 19656.0,
 'profit': 7920.5,
 'rome': 0.6749179838950193}

Strategy B: Basic logistic

In [ ]:
cutoff = break_even  # 0.08333

X_basic = df[['purch']]
y = df['buyer']

logit_basic = LogisticRegression(max_iter=1000)
logit_basic.fit(X_basic, y)

df['p_basic'] = logit_basic.predict_proba(X_basic)[:, 1]
target_basic = df['p_basic'] >= cutoff

basic_results = campaign_metrics(df, target_basic, cost_per_mail, profit_per_sale)
basic_results


{'mailed': 17926,
 'buyers_mailed': 2346,
 'cost': 8963.0,
 'sales': 14076.0,
 'profit': 5113.0,
 'rome': 0.570456320428428}

Strategy C: Enhanced logistic with decision-tree-guided predictors + EXACTLY ONE interaction (NOT the example)

In [ ]:
# predictors 
candidate_cols = [c for c in
                  ['purch','last','total_spent','child','youth','cook','do_it','refernce','art','geog','gender','state']
                  if c in df.columns]

X_tree = df[candidate_cols].copy()

# Encode categoricals for sklearn
cat_cols = [c for c in ['gender','state'] if c in X_tree.columns]
if cat_cols:
    X_tree = pd.get_dummies(X_tree, columns=cat_cols, drop_first=True)

X_tree = X_tree.apply(pd.to_numeric, errors='coerce')
mask = X_tree.notna().all(axis=1)
X_tree = X_tree.loc[mask]
y_tree = df.loc[mask, 'buyer']

X_train, X_test, y_train, y_test = train_test_split(
    X_tree, y_tree, test_size=0.30, random_state=42, stratify=y_tree
)

dt = DecisionTreeClassifier(max_depth=5, min_samples_leaf=100, random_state=42)
dt.fit(X_train, y_train)

importance = (pd.Series(dt.feature_importances_, index=X_train.columns)
              .sort_values(ascending=False))

print("Top 15 Decision Tree Importances:")
print(importance.head(15))


Top 15 Decision Tree Importances:
art         0.595265
last        0.244398
geog        0.092856
gender_M    0.031581
do_it       0.029295
cook        0.006606
purch       0.000000
state_ME    0.000000
state_VI    0.000000
state_VA    0.000000
state_RI    0.000000
state_PA    0.000000
state_NY    0.000000
state_NJ    0.000000
state_NH    0.000000
dtype: float64


Choose predictors + build ONE new interaction

In [ ]:
# Choose predictors + build ONE new interaction
# Create a categorical version of recency: top 20% most recent buyers
df['recency_rank2'] = (-df['last']).rank(method='first')
df['recency_q'] = pd.qcut(df['recency_rank2'], 5, labels=[1,2,3,4,5]).astype(int)
df['recent_top'] = (df['recency_q'] == 5).astype(int)  # 1 = most recent quintile

# Ensure geog is numeric 0/1
df['geog'] = pd.to_numeric(df['geog'], errors='coerce').fillna(0).astype(int)

# Interaction term 
# Does being recently active matter more for customers with geography interest when predicting Florence-book purchase?
df['recent_x_geog'] = df['recent_top'] * df['geog']


# Fit enhanced logistic (main effects + the one interaction) 

In [106]:
enh_cols = [c for c in ['purch','total_spent','recent_top','geog','recent_x_geog'] if c in df.columns]


X_enh = df[enh_cols].copy()
X_enh = X_enh.apply(pd.to_numeric, errors='coerce')

mask_enh = X_enh.notna().all(axis=1)
X_enh = X_enh.loc[mask_enh]
y_enh = df.loc[mask_enh, 'buyer']

logit_enh = LogisticRegression(max_iter=2000)
logit_enh.fit(X_enh, y_enh)

df.loc[mask_enh, 'p_enh'] = logit_enh.predict_proba(X_enh)[:, 1]
target_enh = df['p_enh'] >= cutoff

enh_results = campaign_metrics(df, target_enh, cost_per_mail, profit_per_sale)
enh_results


{'mailed': 19428,
 'buyers_mailed': 2855,
 'cost': 9714.0,
 'sales': 17130.0,
 'profit': 7416.0,
 'rome': 0.7634342186534898}

In [ ]:
# Compare all three strategies in one table
results_table = pd.DataFrame([
    {"strategy":"RFM", **rfm_results},
    {"strategy":"Basic Logistic (buyer purch)", **basic_results},
    {"strategy":"Enhanced Logistic (+1 interaction)", **enh_results},
])

# Optional rounding
results_table['rome'] = results_table['rome'].round(3)
results_table['profit'] = results_table['profit'].round(2)
results_table['cost'] = results_table['cost'].round(2)
results_table['sales'] = results_table['sales'].round(2)

print(results_table.to_string(index=False))


                          strategy  mailed  buyers_mailed    cost   sales  profit  rome
                               RFM   23471           3276 11735.5 19656.0  7920.5 0.675
      Basic Logistic (buyer purch)   17926           2346  8963.0 14076.0  5113.0 0.570
Enhanced Logistic (+1 interaction)   19428           2855  9714.0 17130.0  7416.0 0.763


After comparing the three strategies, the enhanced logistic regression achieves the highest ROME, indicating the most efficient use of marketing spend. While RFM produces slightly higher total profit due to mailing a larger audience, it does si at lower return per dollar spent. The enhanced logistic model improves efficiency by incorporting additional behavioral predictors and an interaction between receny and geography affiniyy, allowing it to concentrate mailing on customers with both high engagement and topic fit. Overall, this demonstrates how richer mdoels and interactions terms enable more precise targeting and higher profitability efficicency.